<a href="https://colab.research.google.com/github/ced-sys/ENHANCING-SEISMIC-EVENT-DETECTION-USING-RECURRENT-NEURAL-NETs/blob/main/ENHANCING_SEISMIC_EVENT_DETECTION_USING_RECURRENT_NEURAL_NETs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies
##Purposes
This code installs Python Libraries necessary for the project for machine learnin, signal processing and data visualization

#Libraries
1. Tensorflow
An open source machine learning framework for creating and deploying deep learning mmodels

#Obspy
Python library designed for processing and analysing seismological data.

It provides tools fro reading, writing and processing waveform data and supports seismic event analysis.

#Matplotlib
Matplotlib is a plotting library used for creating static, interactive and animated visualizations.

It helps visualize seismic waveforms, and machine learning model outputs.

#Scikit-learn
Machine learning library that provides tools for classification and providing the metrics of scoring the model.

It is also used in feature engineering.


In [2]:
!pip install tensorflow obspy matplotlib scikit-learn

# Import Libraries and modules
## Core libraries
`numpy` and `matplotlib.pyplot`: For numerical operations and data visualization.
`os`: For file and directory management.

## Seismology tools
`obspy`: Manages seismic data, including date/time handling (`UTCDateTime`), data streams (`Stream`, `Trace`), and fetching data from online servers (`Client`). It also includes signal processing methods like `classic_sta_lta` for detecting seismic events and `bandpass` for filtering.

## Machine Learning Tools
`sklearn`: Provides tools for data splitting (`train_test_split`, `StratifiedKFold`), scaling (`MinMaxScaler`, `StandardScaler`), and performance evaluation metrics (precision, recall, F1, confusion matrix, ROC curve)

##Deep Learning Tools
`tensorflow.keras`: For creating and training RNN models. Key components include:

*   **Model Building**: `Sequential`
*   **RNN Layers**: `LSTM`, `GRU`, `SimpleRNN`, `Bidirectional`
*   **CNN Layers**: `Conv1D`, `MaxPooling1D`
*   **Regularization and Optimization**: `l2`, `Adam`
*   **Training Callbacks**: `EarlyStopping`, `ReduceLROnPlateau`, `ModelCheckpoint`.

##Visualization Tools
`seaborn`: For advanced statistical plotting.
`tensorflow`: Core library for building and deploying machine learning models.



In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
from obspy import UTCDateTime, Stream, Trace
from obspy.clients.fdsn import Client
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.filter import bandpass
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (LSTM, Dense, Dropout, BatchNormalization,
                                     Bidirectional, SimpleRNN, GRU, Conv1D, MaxPooling1D)
from tensorflow.keras.callbacks import(
    EarlyStopping, ReduceLROnPlateau,
    ModelCheckpoint
)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    classification_report, confusion_matrix,
    accuracy_score, roc_auc_score, roc_curve,
    precision_recall_curve
)
import seaborn as sns
import tensorflow as tf


# Create output directory
The function `create_output_directory()` creates a directory named `seismic_detection_outputs` for saving plots, models or any other project related files.

##Define directory name
`base_dir='seismic_detection_outputs'`sets the directory name

##Create directory
`os.makedirs(base_dir, exist_ok=True)`creates the directory if it doesn't already exist. The `exist_ok=True` parameter prevents errors if the directory already exists.

##Return Directory Path
The function returns the directory path `base_dir` or later use in saving files.

In [4]:
def create_output_directory():
  """Create a directory for saving plots and models"""
  base_dir='seismic_detection_outputs'
  os.makedirs(base_dir, exist_ok=True)
  return base_dir

# Fetch IRIS data
`network`: Seismic network code

`station`: Seismic station code

`location`: Station location identifier

`channel`:Channel code for seismic data

`origin_time`: Start time for fetching, in UTC format

`duration`: Length of time (in seconds) to fetch data after `origin_time`.

##Function workflow
1. **Initialize Client**: `client=Client("IRIS")` sets up a connection to the IRIS database.

2. **Define time range**: `start_time=origin_time` sets the starting point. `end_time=origin_time+duration` calculates the ending point.

3. **Fetch Seismic data**: `stream=client.get_waveforms()` fetches seismic waveform data based on the provided parameters.

4. **Handle errors**: If the data fetching process fails, the function prints an error message and returns `None`.

##Return Value
Returns the `stream` object containing the seismic waveform data if successful or `None` if an error occurs.


In [5]:
def fetch_iris_data(network, station, location, channel, origin_time, duration=120):
  """Fetch Seismic data from the IRIS database"""
  try:
    client=Client("IRIS")
    start_time=origin_time
    end_time=origin_time+duration

    stream=client.get_waveforms(network, station, location, channel, start_time, end_time)
    return stream
  except Exception as e:
    print(f"Error fetching data from IRIS:{str(e)}")
    return None


# Generate Synthetic Data
The function creates synthetic seismic waveform data, including background noise and multiple seismic-like events.

##Function parameters
`num_samples`: Total number of samples in the synthetic signal.

`sample_rate`: Number of samples per second.

`event_duration`: Length of each seismic event in seconds.

`noise_level`: Standard deviation of the background noise.

`num_events`: Maximum number of events to generate (default is 3).

##Function workflow
1. **Initialize variables**:

`time`: Array representing the time points of the synthetic signal.

`background`: Random nise with mean 0 and standard deviation `noise_level`.

`events`: Zero-initialized array for storing events.

`event_locations`: List to store start and end indices of generated events.


2. **Generate seismic events**:
Loop through a random number of events (between 1 and `num_events`).

**Event location**:Randomly select an event start index between 1/8 and 7/8 of the signal to avoid edge effects.

Calculate the event's and index based on `event_duration`.

**Event Characetristics**: Randomly generate frequencies (`freq1`, `freq2`), amplitudes (`amp1`, `amp2`) and decay rates (`decay1`, `decay2`) for two sine wave components.

Create the event as a decaying sinusoidal combination to simulate realistic seismic waveforms.

**Insert Event**: Add the generated event to the corresponding range in the `events` array.

3.  **Combine Background and Events**: add `background` and `events` to create the final synthetic data.

##Return Values
`data`: Combined synthetic waveform (background + events)
`events`: Isolated events without noise.
`event_locations`: List of tuples indicating event start and end indices.




In [6]:
def generate_synthetic_data(num_samples, sample_rate, event_duration, noise_level, num_events=3):
  """Generate synthetic data with multiple events"""
  time=np.arange(num_samples)/ sample_rate
  background=np.random.normal(0, noise_level, num_samples)
  events=np.zeros(num_samples)
  event_locations=[]

  for _ in range(np.random.randint(1, num_events+1)):
    event_start=np.random.randint(num_samples // 8, num_samples*7//8)
    event_end=event_start + int(event_duration*sample_rate)
    event_locations.append((event_start, event_end))

    #Create more realistic events with multiple frequency components
    freq1=np.random.uniform(3,8)
    freq2=np.random.uniform(10, 20)
    amp1=np.random.uniform(1.0, 2.0)
    amp2=np.random.uniform(0.5, 1.5)
    decay1=np.random.uniform(0.1, 0.3)
    decay2=np.random.uniform(0.2, 0.4)

    event_time=time[event_start:event_end]-time[event_start]
    event=(
        amp1*np.sin(2*np.pi*freq1*event_time)*np.exp(-event_time/decay1)+
        amp2*np.sin(2*np.pi*freq2*event_time)*np.exp(-event_time/decay2)
    )
    events[event_start:event_end]=event
  data=background+events
  return data, events, event_locations

# Process Stream
This function performs advanced preprocessing on a seismic data stream to enhance signal quality and detect seismix events using the STA/LTA method.

## Function parameters
`stream`: The seismic data stream to process.
`freqmin`: Minimum frequency for bandpass filtering (default is 0.5 Hz).
`freqmax`: Maximum frequency fo bandpass filtering (default is 20 Hz).

## Processing Workflow
1. **Create a copy**: `processed_stream=stream.copy()` avoids modifying the original data.

2. **Preprocessing Steps**:

`detrend('linear')`:Removes linear trends from the data

`taper(max_percentage=0.1)`: Applies a 10% taper to reduce edge effects.

`filter ('bandpass'...)`: Applies a 6th-order zero-phase bandpass filter to isolate relevant seismic frequencies.

3. **STA/LTA Trigger Calculation**:

Extract the first trace: `trace=processed_stream[0]`

Apply the STA/LTA algorithm `classic_sta_lta(trace.data, int(0.5*trace.stats.sampling_rate), int(10*trace.stats.sampling_rate))` computed the characteristic function `cft`.

The short-term average (STA) window is 0.5 seconds.

The long-term average (LTA) window is 10 seconds.

##Return values
`processed_stream`: The preprocessed seismic stream.
`cft`: The STA/LTA characteristic function for event detection.

In [7]:
def process_stream(stream, freqmin=0.5, freqmax=20):
  """Enhanced stream processing with more robust filtering"""
  processed_stream=stream.copy()
  processed_stream.detrend('linear')
  processed_stream.taper(max_percentage=0.1)
  processed_stream.filter('bandpass', freqmin=freqmin, freqmax=freqmax,
                          corners=6, zerophase=True)

  #Add STA/LTA trigger for additional event detection
  trace=processed_stream[0]
  cft=classic_sta_lta(trace.data, int(0.5*trace.stats.sampling_rate),
                      int(10*trace.stats.sampling_rate))
  return processed_stream, cft

# Enhanced windows
The function generates labelled data windows with advanced feature engineering for seismic event prediction. It uses synthetic event location, STA/LTA triggers and statistical anomaly detection for labelling.

##Function Parameters
`data`: The input seismic data array.

`window_size`: Number of samples per window.

`step`: Step size for moving the window along the data.

`event_locations`: List of event start and end indices from synthetic data (optional).

`cft`: STA/LTA characteristic function for event detection (optional)


##Inner Function
`detect_event_by_trigger(window_start, window_end, cft)`
Checks if the maximum STA/LTA value within a window exceeds a threshold (3.0).

Returns `True` if an event is detected, otherwise `False`.

##Processing Workflow

1. **Initialize Lists**:
`windows=[]`: To store windowed features.

`labels=[]`: To store corresponding labels.

2. **Loop through data**:

Sliding window from start to end of `data` with the specified `step`.

Extract a data `window` of size `window_size`.

3. **Feature Engineering**:

**Time Domain Features**:
*  Raw signal (`window`).
*  Absolute amplitude (`np.abs(window)`)
*  First and second deravitives (`np.gradient(window)`), `np.gradient(np.gradient(window))`)
*  Log-transformed absolute amplitude (`np.log1p(np.abs(window))`).

**Frequency Domain features**
Compute the real part of the FFT (`np.fft.fft(window)`), keeping half of the spectrum.

Pad the FFt result to match `window_size`.

Stack all features into `window_features`.

4. **Labelling logic**:
Initialize `label=0` (no event detected).

**Synthetic event check**
Check if window overlaps with any event in `event_locations`.

**STA/LTA Trigger check**:
If no synthetic event is found, use STA/LTA detection.

**Anomaly Detection (Fallback)**:
If still no event, compare the window's mean absolute value to the data's standard deviation.

Assign `label=1`if the mean exceeds 2.5 times the standard deviation,

5. **Store results**

Append `window_features` to `windows`.
Append `label` to `labels`.

##Return Values

`windows`: A 2D array of windowed features for model input.
`labels`: Corresponding labels (0 for no event, 1 for event).




In [8]:
def create_enhanced_windows(data, window_size, step, event_locations=None, cft=None):
  """Create window with advanced feature engineering and labelling"""
  windows=[]
  labels=[]

  #add STA/LTA trigger-based event detection if available
  def detect_event_by_trigger(window_start, window_end, cft=None):
    if cft is not None:
      window_cft=cft[window_start:window_end]
      return np.max(window_cft)>3.0 #Adjust threshold as needed
    return False

  for i in range(0, len(data)-window_size+1, step):
    window=data[i:i+window_size]
    freq_features=np.fft.fft(window)[:window_size//2].real
    freq_features_padded=np.pad(freq_features, (0, window_size-len(freq_features)), 'constant')

    #Advanced feature engineering
    window_features=np.column_stack([
        window, #Raw signal
        np.abs(window), #Absolute amplitude
        np.gradient(window), #First deravitive
        np.gradient(np.gradient(window)), #Second derivative
        np.log1p(np.abs(window)), #Log of absolute amplitude
        freq_features_padded, #Padded frequency domain features
    ])

    #Labelling logic with multiple detection methods
    label=0
    if event_locations:
      #Check if window contains an event from synthetic data
      for start, end in event_locations:
        if (i<=end and i +window_size >= start):
          label=1
          break

    #Additional event detection using STA/LTA
    if label==0 and detect_event_by_trigger(i, i +window_size, cft):
      label=1

    #Fall back: statistical anomaly detection
    if label==0:
      window_mean=np.abs(window).mean()
      data_std=np.abs(data).std()
      label=1 if window_mean> data_std *2.5 else 0

    windows.append(window_features)
    labels.append(label)
  return np.array (windows), np.array(labels)

# Build advanced RNN model
The function defines a deep learning model combining Convolutional Neural Netwroks (CNNs) and Recurrent Neural Networks (RNNs) for seismic event prediction. This hybrid architecture leverages both spatial and temporal feature extraction techniques.

##Function Parameter
`input_shape`: Shape of the input data, typically `(window_size, num_features)`.

##Model Architecture:

1. **Convolutional Layer for Feature Extraction**:
`Conv1D(64, kernel_size=5, activation='relu')`: Extracts spatial features with 64 filter and a kernel size of 5.

`MaxPooling1D(pool_size=2)`: Reduces the dimensionality be half.

`BatchNormaliation()`: Normalizes the activations to stabilie and accelerate trainig.

2. **Recurrent Layers for Temporal Dependenices**:
`Bidirectional(GRU(128, return_sequences=True))`: Captures long-term dependencies in both forward and backward directions.
`return_sequences=True` enables stacking further RNN layers.

`BatchNormalization()` and `Dropout(0.4)`: Improves training stability and prevents overfitting.

`Bidirectional(LSTM(96, return_sequences=True))`: Similar to the GRU layer but with increased model complexity.

`SimpleRNN(64)`: Adds a final RNN layer to extract additional sequential patterns.

3. **Dense Layers for Classification**:
`Dense(128, activation='relu)`: fully connected layer with ReLU activation and L2 regularization

`BatchNormalization()` & `Dropout(0.3)`: Stabilizes training and reduces overfitting.

`Dense(64, activation='relu)`: Second fully connected layer for deeper feature integration.

`Dense(1, activation='sigmoid')`: Final output layer for binary classification (seismic event or no event).

##Optimizer and Compilation
**Optimizer**: `Adam(learning_rate=0.0003, beta_1=0.9, beta_2=0.999)` is configured for more aggressive optimization with a low learning rate.

**Loss function**: `binary_crossentropy` is used for binary classification task.

**Metrics**: `accuracy`,`Precision()` and `Recall()` monitor the model's performance comprehensively.

##Return Value:
The compiled model is returned, rady for training and evaluation


In [9]:
def build_advanced_rnn_model(input_shape):
  """Enhanced RNN model with CNN-RNN hybrid architecture"""
  model= Sequential([
      #Convolutional layer for feature extraction
      Conv1D(64, kernel_size=5, activation='relu', input_shape=input_shape,
             kernel_regularizer=l2(0.001)),
      MaxPooling1D(pool_size=2),
      BatchNormalization(),


  #Bidirectional RNN layers with increased complexity
  Bidirectional(GRU(128, return_sequences=True,
                    kernel_regularizer=l2(0.001))),
  BatchNormalization(),
  Dropout(0.4),

  Bidirectional(LSTM(96, return_sequences=True,
                     kernel_regularizer=l2(0.001))),
  BatchNormalization(),
  Dropout(0.4),

  #Additional RNN layer
  SimpleRNN(64, kernel_regularizer=l2(0.001)),
  BatchNormalization(),
  Dropout(0.3),

  #Dense layers for classification with increased regularization
  Dense(128, activation='relu', kernel_regularizer=l2(0.002)),
  BatchNormalization(),
  Dropout(0.3),

  Dense(64, activation='relu', kernel_regularizer=l2(0.002)),
  Dense(1, activation='sigmoid')

  ])

  #More aggresive optimizer configuration
  optimizer=Adam(
      learning_rate=0.0003,
      beta_1=0.9,
      beta_2=0.999
  )

  model.compile(
      optimizer=optimizer,
      loss='binary_crossentropy',
      metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]

  )

  return model

# Train and evaluate the model
This function trains and evaluates the RNN model using cross-validation, with various enhancements for model training, evaluation and performance tracking

##Function Parameters
`X`: Input feature for training, shaped as `(num_samples, num_timesteps, num_features)`.

`y`: Labels for the samples, either 0 (no event) or 1 (event).

`base_dir`: directory to save model checkpoints, plots, and evaluation results.

##Function Workflow:
1. **Feature Scaling**:
The function uses `StandardScaler() to scale the input features (`X`) to have a mean of 0 and a standard deviation of 1 for improved model performance.

`X_scaled=scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)`reshapes and scales the input data accordingly.

2. **Cross-Validation setup**:
**Stratified K-Fold Cross Validation**: `StratifiedKFold(n_splits=10)` ensures that each fold has a balanced distribution of classes (seismic event and no event).

The cross-validation splits the data into 10 folds.

3. **Model training and evaluation**:For each fold:

**Training and Validation split**: The data is divided into training (`X_train, y_train`) using the indices provided by `StratifiedKFold`.

**Model Initialization**:
The RNN model is re-initialized for each fold to avoid training from previous fold weights.

**Callbacks**:
Several callbacks are used for better training management:
*  **EarlyStopping: Stops training if the validation loss doesn't improve after 20 epochs, restoring the best weights.

*  **ReduceLROnPLateau**: Reduces the learning rate by a factor of 0.3 if the validation loss doesn't improve for 10 epochs.

*  **ModelCheckpoint**: Svaes the best model based on the validation recall score.

**Model training**:
The model is trained for a maximum of 150 epochs with a batch size of 32. The `class_weight={0:1., 1:3.}` argument assigns more weight to the event class (class`1`), compensating for class imbalance.

**Predictions and Metrics**:
The model's predictions are made on the validation set, and binary classification is performed by thresholding the predicted probabilities at 0.5.

Metrics calculated include:
1.  Accuracy
2.  Precision
3.  Recall
4. F1 score
5. AUC(Area Under the Curve)

**Plots**:
Several plots are generated for performance visualization:
*  **ROC Curve**(`plot_roc_curve`)
*  **Training history**(`plot_training_history`)
*  **Confusion Matrix**(`plot_confusion_matrix`)
*  **Precision-Recall Curve**(`plot_precision-recall_curve`)

4. **Cross-validation results**:
After completing all the folds, the function prints the average cross-validation score for each metric (accuracy, precision, recall, F1 Score and AUC) with their standard deviations.

##Return values
`model`: The trained model from the final fols, saved after training.
`cv_scores`: A dictionary containing cross-validation scores for eac metric accuracy, precision, recall, F1 Score and AUC

##Key enhancements
**Cross-Validation**: Using Stratified K-Fold ensures balabced training and testing sets.

**Regularization**: includes L2 regularization, dropout, and batch normalization to improve generalization and avoid overfitting.

**Class weighing**: More emphasis on detecting seismic events by applying stronger weight to class `1`.

**Comprehensive Evaluation**: Tracks a variety of performance metrics, generates detailed reports and visualizes results through multiple plots.



In [10]:
def train_and_evaluate_model(X, y, base_dir):
  """Comprehensive model training with cross-validation"""
  #Use StandardScaler for better feature scaling
  scaler=StandardScaler()
  X_scaled=scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

  #Stratified K-Fold cross validation with more splits
  kfold=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

  #Tracking metrics
  cv_scores={
      'accuracy':[], 'precision':[],
      'recall':[], 'f1':[], 'auc':[]
  }

  #Model checkpoint directory
  model_dir=os.path.join(base_dir, 'best_models')
  os.makedirs(model_dir, exist_ok=True)

  for fold, (train_idx, val_idx) in enumerate(kfold.split(X_scaled, y), 1):
    print(f"\n=== Fold {fold}===")

    X_train, X_val=X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val=y[train_idx], y[val_idx]

    #Reset model for each fold
    model=build_advanced_rnn_model((X.shape[1], X.shape[2]))

    #Improved callbacks
    callbacks=[
        EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,
            patience=10,
            min_lr=1e-6
        ),
        ModelCheckpoint(
            filepath=os.path.join(model_dir, f'best_model_fold_{fold}.keras'),
            monitor='val_recall',
            mode='max',
            save_best_only=True
        )
    ]

    #Train model with more aggressive class weighting
    history=model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=150,
        batch_size=32,
        callbacks=callbacks,
        class_weight={0:1., 1: 3.}, #Stronger emphasis on event class
        verbose=1
    )

    #Predictions and evaluation
    predictions=model.predict(X_val).flatten()
    binary_predictions=(predictions >0.5).astype(int)

    #compute metriics
    accuracy=accuracy_score(y_val, binary_predictions)
    precision=precision_score(y_val, binary_predictions)
    recall=recall_score(y_val, binary_predictions)
    f1=f1_score(y_val, binary_predictions)
    auc=roc_auc_score(y_val, predictions)

    cv_scores['accuracy'].append(accuracy)
    cv_scores['precision'].append(precision)
    cv_scores['recall'].append(recall)
    cv_scores['f1'].append(f1)
    cv_scores['auc'].append(auc)

    #Print detailed classification report
    print("\nClassification Report:")
    print(classification_report(y_val, binary_predictions))

    #Plot ROC curve
    plot_roc_curve(y_val, predictions, base_dir, fold)
    plot_training_history(history, base_dir)
    plot_confusion_matrix(y_val, binary_predictions, base_dir)
    plot_precision_recall_curve(y_val, predictions, base_dir)

  #Print average cross-validation scores
  print("\n=== Cross-Validation Results ===")
  for metric, scores in cv_scores.items():
    print(f"{metric.capitalize()}: {np.mean(scores):.4f} (±{np.std(scores):.4f})")

  return model, cv_scores


# Plot Receiver Operating Characteristic
The ROC curve visualizes the trade-off between the true positive rate and the false positive rate at different thresholds, helping evaluate the classifier's performance. A curve closer to the top-left corner indicates better performance.

## Function definition and inputs
`y_true`: Actual labels (ground truth).
`y_scores`: Predicted probabilities from the model.
`base_dir`: Directory where the ROC curve image will be saved.
`fold`: The current fold number in cross-validation.

##Compute ROC metrics
`fpr, tpr, thresholds=roc_curve(y_true, y_scores):`

`fpr`: False Positive Rate.
`tpr`: True Positive Rate.
`thresholds`: Decision thresholds used to compute `fpr` and `tpr`.

##Plot the ROC Curve


*   Sets the figure size to 8*6 inches
*   Plots the ROC curve using `fpr` and `tpr`.
*   Adds a diagonal red dashed line representing a random classifier (baseline).
*   Adjusts the axis limits for better visualization.
*   Labels the axes and titles the plot.
*   Adds a legend to distinguish the model from the baseline.

##Save the Plot
*  Constructs the file path using `os.path.join`.
*  Saves the plot as `roc_curve_fold_{fold}.png` in the specified directory.
*  Closes the plot to free up memory and avoid overlapping features.



In [11]:
def plot_roc_curve(y_true, y_scores, base_dir, fold):
  """Plot and save ROC curve"""
  fpr, tpr, thresholds=roc_curve(y_true, y_scores)

  plt.figure(figsize=(8,6))
  plt.plot(fpr, tpr, color='blue', label='ROC curve')
  plt.plot([0,1], [0,1], color='red', linestyle='--', label='Random Classifier')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title(f'Receiver Operating Characteristic- Fold{fold}')
  plt.legend(loc='lower right')

  #Save plot
  plt.savefig(os.path.join(base_dir, f'roc_curve_fold_{fold}.png'))
  plt.close()

# Plot training history
This function visualizes the training and validation performance metrics from the model's training from the model's training history and saves the generated plots as an image file.

##Function Definition and inputs
`history`: Training history object returned by `model.fit()`.
`base_dir`: Directory where the training metrics plot will be saved.

##Figure Initialization
Sets the figure size to `15*10` inches.

##Subplot 1: Model Loss
Plots training hand validation loss values from `history.history['loss']` and `history.history['val_loss']`.

Labels the axis as "Epoch" and "Loss".

Adds a title "Model Loss" and a legend to differentiate between training and validation loss.

##Subplot 2: Model Accuracy
Plots training and validation accuravy from `history.history['accuracy']` and `history.history['val_accuracy']`.

Labels the axes as 'Epoch' and "Accuracy".

Adds a title "Model Accuracy" and a legend to sitinguish between training and validation accurcay.

##Layout adjustment and saving
Uses `plt.tight_layout()` to optimize subplot spacing.

Saves the figure as `training_metric.png` in the specifies `base_dir`.

Closes the figure to prevent memory issues or overlapping plots.


In [12]:
def plot_training_history(history, base_dir):
  """Plot Training and validation metrics"""
  plt.figure(figsize=(15, 10))

  plt.subplot(2,2,1)
  plt.plot(history.history['loss'], label='Training Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  plt.title("Model Loss")
  plt.xlabel('Epoch')
  plt.ylabel('loss')
  plt.legend()

  plt.subplot(2,2,2)
  plt.plot(history.history['accuracy'], label='Training Accuracy')
  plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
  plt.title("Model Accuracy")
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend()

  plt.tight_layout()
  plt.savefig(os.path.join(base_dir, 'training_metric.png'))
  plt.close()

# Plot Confusion Matrix
The function generates a heatmap of the confusion matrix from predicted and true labels,providing insight into the model's classification performance. The resulting plot is saved as an image file.

##Function definition and inputs
`y_true`: Actual labels from the validation dataset.

`y_pred`: Predicted labels generated by the model.

`base_dir`: Directory where the confusion matrix plot will be saved.

##Compute Confusion Matrix
Calss `confusion_matrix(y_true, y_pred)` to calculate the confusion matrix, which shows the number of correct and incorrect predictions for each class.

##Create Heatmap plot
Initializes a plot of size 8*6 inches.

Uses `sns.heatmap()` to create a heatmap visualization.

`cm`: confusion matrix data.

`annot=True`: Displays numeric values on the heatmap.

`fmt=d` formats annotations as integers.

`cmap=Blues`: Sets the heatmap color theme.

`xticklabels` and `yticklabels`: Labels for predicted and true classes.

##Add titles and Labels
Sets the plot title to "Confusion Matrix".

Labels the x-axis as "Predicted Label."

Labels the y-axis as "True Label."

##Save the plot
Adjusts layout with `plt.tight_layout()` to avoid clipping.

Saves the figure as `confusion_matrix.png` in `base_dir`.

Closes the figure to prevent overlapping or memory issues.





In [13]:
def plot_confusion_matrix(y_true, y_pred, base_dir):
  cm=confusion_matrix(y_true, y_pred)
  plt.figure(figsize=(8,6))
  sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
              xticklabels=['No Event', 'Event'],
              yticklabels=['No Event', 'Event'])
  plt.title('Confusion Matrix')
  plt.xlabel('Predicted Label')
  plt.ylabel('True label')
  plt.tight_layout()
  plt.savefig(os.path.join(base_dir, 'confusion_matrix.png'))
  plt.close()



# Plot Precision Recall Curve
The function generates a Precision-Recall Curve which is a graphical representation of the tradeoff between precision and recall for different classification thresholds. It saves the plot as an image file in the specified directory.

##Parameters
`y_true`: True binary labels of the dataset(0, 1), indicating whether a data point belongs to the positive case.

`y_scores`: Predicated probabilities or scores are assigned to the positive class by the classifier. These scores are used to compute precision adn recall at various thresholds.

`base_dir`: Directory path where the generated plot will be saved.

##Functionality
1. **Computing Precision and Recall**
The function uses `precision_recall_curve` from `sklearn.metrics` to compute precision and recall values over a range of thresholds.

The function also outputs thresholds

2. **Plotting the curve**
A new figure is created with a size of `8*6` inches for clear visualization.

Precision is plotted on the y-axis and recall is plotted on the x-axis. A blue line represents the curve.

Labels and a legend are added to make the plot informative.

3. **Adjusting the Layout**
The `plt.tight_layout()` method ensures that the elements of the plot are arranged neatly without overlap.

4. **Saving the Plot**
The plot is saved as an image named `precision_recall_curve.png` in the specified `base_dir` using `os.path.join` for robust path handling.

5. **Closing the plot**
`plt.close()` is called to release resources and avoid displaying of the figure.

In [14]:
def plot_precision_recall_curve(y_true, y_scores, base_dir):
  precision, recall, _=precision_recall_curve(y_true, y_scores)

  plt.figure(figsize=(8,6))
  plt.plot(recall, precision, color='blue', label='Precision-Recall Curve')
  plt.title('Precision-Recall curve')
  plt.xlabel('Recall')
  plt.ylabel('Precision')
  plt.legend()
  plt.tight_layout()
  plt.savefig(os.path.join(base_dir, 'precision_recall_curve.png'))
  plt.close()

# Plot Seismic Data
The function is designed to visualize and compare seismi data from three diffeerent sources, the real seismic data, synthetic data and a combination of both. It generates a side-by-side comparison of these data sources in the form of a plot and saves the pot as an image file

##Parameters
`real_data`: The actual seismic data from Indonesian station.

`synthetic_data`: Simulated seismic data generated through modelling or computational techniques.

`combined_data`: Data resulting from the combination of real and synthetic data, often used to analyze correlation or validate models.

`base_dir`: The directory path where the plot image will be saved.

##Functionality
1. **Figure Setup**
The function creates a `matplotlib` figure with a size of `15*10` inches, providing ample space for three subplors stacked vertically.

2. **Subplots for Visualization**
First Subplot: Plots the `real_data` as "Real Seismic Data" with x-axis for samples and y-axis for amplitude.

Second Subplot: Plots the `synthetic_data` array. Labels the plot as "Synthetic Seismic Data" with the same axis labels.

Third Subplot: Plots the `combined_data`/ Labels the plot as "Combined Seismic Data", maintaining the labels.

3. **Layout Adjustments**
The function uses `plt.tight_layout()` to ensure there is no overlapping of subplot titles or axes.

4. **Saving the Plot**
The plot is saved as `seismic_data_comparison.png` in the specified `base_dir` directory using the `os.path.join` method.

5. **Closing the Plot**
`plt.close()` ensures the figure is properly closed, freeing up memory resources and avoiding display in the environment.


In [15]:
def plot_seismic_data(real_data, synthetic_data, combined_data, base_dir):
  plt.figure(figsize=(15, 10))

  plt.subplot(3,1,1)
  plt.plot(real_data)
  plt.title('Real Seismic Data')
  plt.xlabel('Samples')
  plt.ylabel('Amplitude')

  plt.subplot(3,1,2)
  plt.plot(synthetic_data)
  plt.title('Synthetic Seismic Data')
  plt.xlabel('Sample')
  plt.ylabel('Amplitude')

  plt.subplot(3,1,3)
  plt.plot(combined_data)
  plt.title('Combined Seismic Data')
  plt.xlabel('Sample')
  plt.ylabel('Amplitude')

  plt.tight_layout()
  plt.savefig(os.path.join(base_dir, "seismic_data_comparison.png"))
  plt.close()

#The Main Function
The `main` function orchestrates the complete workflow for processing seismic data, generating synthetic data, combining datasets, and training a model detect seismic events. It integrates data preparation, visualization and machinelearning in a sequential and modular manner.

##Steps and functionality
1. **Configuration Setup**
Defines a configuration dictionary `config` with parameters for:

  `window_size`: Size of the data window for training.

  `step`: Step size for sliding window operations.

  `sample_rate`: Default sampling rate (Hz).

  `num_samples`: Number of samples in fallback synthetic data.

2. **Output Directory Creation**
Calls `create_output_directory` to create a directory for saving all outputs, including plots and model files.

3. **Fetching real earthquake data**
Defines the earthquake's origin time using `UTCDateTime` and fetches data using `fetch_iris_data` with:

  Seismic network, station and channel information.

  `origin_time` for temporal alignment.

  Duration of 120 seconds.

If the data fetch is successful:

   Processes the data stream using `process_stream` to filter the signal and compute the charcteristic function (`real_cft`).

  Extracts raw data and its sampling rate.

If no data is fetched:

  Initializes fallback data as a zero-filled array with length `num_samples` and defualt `sample_rate`.

  Sets `real_cft` to `None`.

4. **Synthetic Data Generation**
Calls `generate_synthetic_data` to create synthetic seismic data with parameters:

  Length equal to the rel data.

  Sampling rate.

  Event duration (2 seconds) and noise level(0.1)

Outputs include:

  `synthetic_data`: Generated data.

  `synthetic_events`: List of synthetic event times.

  `synthetic_event_location`: Location of synthetic events in the data.

5. **Combining Datasets**:
Adds `real_data` and `synthetic_data` element wise to create `combined_data`, simulating real-world scenarios where noise and real seismic signals overlap.

6. **Creating Windows and labels**
Uses `create_enhanced_windows` to split `combined_data` into sliding widows with:

  Defines `window_size` and `step` from the configuration

  Event locations (`synthetic_event_location`) for labelling.

  The real signal's characteristic function (`real_cft`) to refine labelling.

7. **Model Traing and Evaluation**
Calls `train_evaluate_model` to train the model using:

  Features (`X`) and labels (`y`) from the windowed data.

  Outputs the trained model and cross-validation scores.

8. **Visualization**
Generates a plot comparing `real_data`, `synthetic_data` and `combined_data` using `plot_seismic_data` saving it in the `base_dir`.





In [16]:
def main():
    # Configuration
    config = {
        'window_size': 250,
        'step': 25,
        'sample_rate': 100,
        'num_samples': 25000
    }

    # Create output directory
    base_dir = create_output_directory()

    # Fetch real earthquake data
    origin_time = UTCDateTime("2015-08-11T16:22:15.200000")
    real_stream = fetch_iris_data(
        network='YS', station='BAOP', location='',
        channel='BHZ', origin_time=origin_time, duration=120
    )

    # Process real and synthetic data
    if real_stream is not None:
        filtered_real_stream, real_cft = process_stream(real_stream)
        real_data = filtered_real_stream[0].data
        sample_rate = real_stream[0].stats.sampling_rate
    else:
        # Fallback to default values if no real data
        real_data = np.zeros(config['num_samples'])
        sample_rate = config['sample_rate']
        real_cft = None

    # Generate synthetic data
    synthetic_data, synthetic_events, synthetic_event_locations = generate_synthetic_data(
        len(real_data),
        sample_rate,
        event_duration=2,
        noise_level=0.1
    )

    # Combine datasets
    combined_data = real_data + synthetic_data

    # Create windows and labels
    X, y = create_enhanced_windows(
        combined_data,
        config['window_size'],
        config['step'],
        synthetic_event_locations,
        real_cft
    )

    # Train and evaluate model
    trained_model, cv_scores = train_and_evaluate_model(X, y, base_dir)
    plot_seismic_data(real_data, synthetic_data, combined_data, base_dir)

    print("\nModel training and evaluation complete. Check the output directory for visualizations.")

In [17]:
if __name__=="__main__":
  main()


=== Fold 1===


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 428ms/step - accuracy: 0.6317 - loss: 2.9141 - precision: 0.4740 - recall: 0.2901 - val_accuracy: 0.2500 - val_loss: 1.9969 - val_precision: 0.2222 - val_recall: 0.5000 - learning_rate: 3.0000e-04
Epoch 2/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 220ms/step - accuracy: 0.5695 - loss: 2.9611 - precision: 0.2398 - recall: 0.1695 - val_accuracy: 0.3333 - val_loss: 1.9912 - val_precision: 0.3000 - val_recall: 0.7500 - learning_rate: 3.0000e-04
Epoch 3/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 182ms/step - accuracy: 0.5977 - loss: 2.6112 - precision: 0.4120 - recall: 0.3452 - val_accuracy: 0.3750 - val_loss: 1.9873 - val_precision: 0.3158 - val_recall: 0.7500 - learning_rate: 3.0000e-04
Epoch 4/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 183ms/step - accuracy: 0.6023 - loss: 2.6705 - precision: 0.4231 - recall: 0.4869 - val_accuracy: 0.4167 - val_loss: 1.9868 - val_precision: 0.3333 - val_recall: 0.7500 - learning_rate: 3.0000e-04
Epoch 5/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step 

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 376ms/step - accuracy: 0.4267 - loss: 2.7159 - precision_1: 0.3066 - recall_1: 0.5224 - val_accuracy: 0.5652 - val_loss: 1.9809 - val_precision_1: 0.4167 - val_recall_1: 0.6250 - learning_rate: 3.0000e-04
Epoch 2/150


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - accuracy: 0.5667 - loss: 2.4576 - precision_8: 0.4362 - recall_8: 0.6190 - val_accuracy: 0.3478 - val_loss: 2.0069 - val_precision_8: 0.3333 - val_recall_8: 0.8750 - learning_rate: 3.0000e-04
Epoch 3/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 217ms/step - accuracy: 0.5725 - loss: 2.5307 - precision_8: 0.3953 - recall_8: 0.5782 - val_accuracy: 0.3478 - val_loss: 2.0255 - val_precision_8: 0.3478 - val_recall_8: 1.0000 - learning_rate: 3.0000e-04
Epoch 4/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step - accuracy: 0.6237 - loss: 2.3631 - precision_8: 0.5039 - recall_8: 0.7609 - val_accuracy: 0.3478 - val_loss: 2.0516 - val_precision_8: 0.3478 - val_recall_8: 1.0000 - learning_rate: 3.0000e-04
Epoch 5/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step - accuracy: 0.6442 - loss: 2.3205 - precision_8: 0.4868 - recall_8: 0.7644 - val_accuracy: 0.3478 - val_loss: 2.0779 - val_precision_8: 0.3478 - val_recall_8: 1.0000 - learning_rate: 3.0000e-04
Epoch 6/150
7/7 ━━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 466ms/step - accuracy: 0.5372 - loss: 2.7626 - precision_9: 0.3842 - recall_9: 0.4801 - val_accuracy: 0.5217 - val_loss: 1.9650 - val_precision_9: 0.3846 - val_recall_9: 0.6250 - learning_rate: 3.0000e-04
Epoch 2/150


/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/model_checkpoint.py:302: UserWarning: Can save best model only with val_recall available.
  if self._should_save_model(epoch, batch, logs, filepath):


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - accuracy: 0.5989 - loss: 2.4641 - precision_9: 0.4553 - recall_9: 0.6177 - val_accuracy: 0.4348 - val_loss: 1.9747 - val_precision_9: 0.3529 - val_recall_9: 0.7500 - learning_rate: 3.0000e-04
Epoch 3/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - accuracy: 0.5714 - loss: 2.5032 - precision_9: 0.4183 - recall_9: 0.4931 - val_accuracy: 0.3478 - val_loss: 1.9869 - val_precision_9: 0.3478 - val_recall_9: 1.0000 - learning_rate: 3.0000e-04
Epoch 4/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - accuracy: 0.6354 - loss: 2.3251 - precision_9: 0.4938 - recall_9: 0.7019 - val_accuracy: 0.3478 - val_loss: 1.9966 - val_precision_9: 0.3478 - val_recall_9: 1.0000 - learning_rate: 3.0000e-04
Epoch 5/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step - accuracy: 0.5864 - loss: 2.4228 - precision_9: 0.4673 - recall_9: 0.5808 - val_accuracy: 0.3478 - val_loss: 2.0026 - val_precision_9: 0.3478 - val_recall_9: 1.0000 - learning_rate: 3.0000e-04
Epoch 6/150
7/7 ━━━━━━━━━━━━━━━━